In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
path_to_file = '5_data.csv.zip'

Step 1 <a href='https://lab.karpov.courses/learning/85/module/1056/lesson/9042/26531/117931/'>link</a>
<br>Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:

<br>InvoiceNo — номер транзакции
<br>StockCode — код товара
<br>Description — описание товара
<br>Quantity — количество единиц товара, добавленных в заказ
<br>InvoiceDate — дата транзакции 
<br>UnitPrice — цена за единицу товара
<br>CustomerID — id клиента
<br>Country — страна, где проживает клиент

В данной задаче ссылка для считывания данных лежит в переменной path_to_file.
Импортируйте пандас и прочитайте данные с кодировкой ISO-8859-1. Запишите полученный датафрейм в retail, а названия колонок сохраните в переменную retail_columns.

In [6]:
retail = pd.read_csv(path_to_file, compression='zip', encoding='ISO-8859-1')
retail_columns = retail.columns

Step 2 <a href='https://lab.karpov.courses/learning/85/module/1056/lesson/9042/26531/117932'>link</a>
<br>Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.

In [8]:
retail.duplicated().sum()

5268

In [10]:
retail = retail.drop_duplicates()

Step 3 <a href='https://lab.karpov.courses/learning/85/module/1056/lesson/9042/26531/117933'>link</a>
<br>⭐️ Задание со звёздочкой! ⭐️

Данные содержат в себе записи как об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). 

Сколько всего транзакций отменили пользователи? Каждый отдельный тип продукта считайте за отдельную транзакцию - уникальные номера считать не надо.

In [11]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [14]:
retail.InvoiceNo.str.startswith('C').sum()

9251

Step 4 <a href='https://lab.karpov.courses/learning/85/module/1056/lesson/9042/26531/117934'>link</a>
<br>Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0. В качестве ответа укажите число оставшихся строк.

In [18]:
retail = retail.query('Quantity > 0')

In [20]:
retail.shape

(526054, 8)

Step 5 <a href='https://lab.karpov.courses/learning/85/module/1056/lesson/9042/26531/117935'>link</a>
<br>💀 Сложное задание! 💀

Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).

In [34]:
country = 'Germany'
N = 0.8
germany_orders = retail.query('Country == @country') \
    .groupby(by='CustomerID', as_index=False) \
    .agg({'InvoiceNo' : 'nunique'})

n_transactions = germany_orders.InvoiceNo.quantile(N)

germany_top = germany_orders.query('InvoiceNo > @n_transactions').CustomerID

Step 6 <a href='https://lab.karpov.courses/learning/85/module/1056/lesson/9042/26531/117936'>link</a>
<br>⭐️ Задание со звёздочкой! ⭐️

Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.

Датафрейм записан в retail, нужные id пользователей – в germany_top .

In [43]:
#top_retail_germany = retail[retail.CustomerID.isin(germany_top)]
top_retail_germany = retail.query('CustomerID in @germany_top')

Step 7 <a href='https://lab.karpov.courses/learning/85/module/1056/lesson/9042/26531/117937'>link</a>
<br>Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST? 

Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

In [50]:
top_retail_germany.groupby(by='StockCode', as_index=False)\
    .agg({'InvoiceNo':'nunique'}) \
    .sort_values('InvoiceNo', ascending=False) \
    .head()

,StockCode,InvoiceNo
1157,POST,213
409,22326,52
411,22328,38
453,22423,34
45,20719,30


Step 8 <a href='https://lab.karpov.courses/learning/85/module/1056/lesson/9042/26531/117938'>link</a>
<br>Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.

Отфильтрованные данные записаны в retail.

In [51]:
retail['Revenue'] = retail.Quantity * retail.UnitPrice

In [52]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


Step 9 <a href='https://lab.karpov.courses/learning/85/module/1056/lesson/9042/26531/117939'>link</a>
<br>Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)

In [65]:
top_5 = retail.groupby(by='InvoiceNo', as_index=False) \
    .agg({'Revenue' : 'sum'}) \
    .rename(columns={'Revenue':'TotalRevenue'}) \
    .sort_values('TotalRevenue', ascending=False) \
    .head(5).InvoiceNo.values

print(', '.join(top_5))

581483, 541431, 574941, 576365, 556444
